In [ ]:
import pandas as pd

Read the CSV file. We know by looking at the CSV file that there are two columns that have dates, thus we ```parse_dates```

In [ ]:
#read the date and show the table
df = pd.read_csv('../mvr-data.csv', parse_dates=['DateOfRecord', 'DateOfEvent'])
#df

There is no need to have a separate dataframe for the transormations. It is there just demonstration purposes. Compare ```df``` and ```traffacc``` to see diferences.

Below we add thre new variables (columns): year, month, and day. The final line is, again, not necessary. It is there to show the new dataframe with new variables. The same happens below after 

In [ ]:
traffacc = df
traffacc['Year'] = pd.DatetimeIndex(df['DateOfEvent']).year
traffacc['Month'] = pd.DatetimeIndex(df['DateOfEvent']).month
traffacc['Day'] = pd.DatetimeIndex(df['DateOfEvent']).day
#traffacc

The time is in a separate column in HH:MM format but it is a string. Because we just need the hours, we create a new variable by which only has hours, and is in datetime format.

In [ ]:
#traffacc['Hour'] = pd.to_datetime(traffacc['Hour'], errors='coerce')
#traffacc['Hour'] = traffacc['Hour'].floor('H')
#traffacc['Hour']

In [ ]:
#from datetime import datetime as dt
#from datetime import timedelta as td

#to make datetime format
#def convertdt(val):
    #try:
        #return dt.replace(dt.strptime(val, '%H:%M').time(), minute=0)
    #except Exception:
        #return None
    
#and to reset minutes
#def resetdt(val):
    #return val + td(minutes = -val.minute, seconds = -dt.second)
    #return dt.replace(val, minute=0)
    
    
#traffacc = traffacc.assign(Hours = traffacc.Hour.map(convertdt))
#traffacc = traffacc.assign(Hours = traffacc.Hours.map(resetdt))


#traffacc.Hours

In [ ]:
def hours(val):
    try:
        return int(val.split(':')[0])
    except Exception:
        return None
    
traffacc = traffacc.assign(Hours = traffacc.Hour.map(hours))
#traffacc

We have decided to just look at traffic accidents, so we filter out anything else by quering for ```Event```.

In [ ]:
traffacc = traffacc.query('Event == "сообраќајна незгода"')
#traffacc

We need to order the ```Outcome``` so that the order represents the severity of the injuries.

In [ ]:
from pandas.api.types import CategoricalDtype

cat_type = CategoricalDtype(categories=["непознато", "повреда", "тешка повреда", "животна опасност", "смрт"],
                            ordered=True)

traffacc = traffacc.assign(Outcome = traffacc['Outcome'].astype(cat_type))

#traffacc['Outcome']


We (probably) need the municipalities' names in latin for plotting. To do that in python we also need to instal transliterate.

In [ ]:
#just an example
#from transliterate import translit, get_available_language_codes
#text = "Lorem ipsum dolor sit amet"
#print(translit(text, 'mk'))

But first we need all the municipalities. There is a value for ```Mun``` when the ```Place``` is a village. But otherwise the municipality and the place are the same. 

In [ ]:
traffacc.Mun.fillna(traffacc.Place, inplace=True)
traffacc

#and now transliterate
def latin(val):
    try:
        return (translit(val, 'mk', reversed=True))
    except Exception:
        return None
    
traffacc = traffacc.assign(MunLatin = traffacc.Mun.map(latin))
#traffacc

Now use seaborn to make a scatter plot of traffc accidents by age and time.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np 

#set default aestetics
sns.set()

#plot facet by type of road
g = sns.FacetGrid(
    data=traffacc,
    col='TypeOfRoad',
    hue='Outcome'
)

g.map(plt.scatter, "AgeV1", "Hours")
g.set_axis_labels('Age','Hour of day')
g.add_legend()
g.set(xticks=np.arange(25,100,25), yticks=np.arange(0,25,4))

In [ ]:
import pandas_profiling
pandas_profiling.ProfileReport(traffacc)